In [1]:
import YAM_CoatingBrownian
import EFI_tmm
import numpy as np 
import pandas as pd 
from scipy.integrate import quad

In [ ]:

def reflectivity(n1, n2):
    """
    Calculate the reflectivity at an interface given two refractive indices.
    This is a simplified model and assumes normal incidence.
    """
   
    return ((n1 - n2) ** 2) / ((n1 + n2) ** 2)

def overall_reflectivity(layers):
    """
    Calculate the overall reflectivity of the coating stack.
    Assumes that 'layers' is a list of refractive indices.
    """
    R_total = 1
    for i in range(len(layers) - 1):
        R = reflectivity(layers[i], layers[i + 1])
        R_total *= R  # Assuming independent reflectivities for simplicity
    return R_total


def merit_function(dOpt,materialLayer,materialParams,w_R, w_T, w_E, w_D,E_total,PhysicalThickness,ThermalNoise):
    #set up with default inputs to match aLIGO for testing = this should be modified to allow for varying inputs. 
    
    """
    Calculate the merit function for a given coating configuration.
    """
    
    #n1 = materialParams[np.unique(materialLayer)[0]]['n']
    #n2 = materialParams[np.unique(materialLayer)[1]]['n']
        
    #n_indicies = [n1, n2] * num21 + [1, 2] * num34
    


    if isinstance(ThermalNoise[0]['Frequency'],float):
        difference_array = np.absolute(ThermalNoise[0]['Frequency']-100)
        
        # find the index of minimum element from the array
        index = difference_array.argmin()
        
        ThermalNoise_Total = ThermalNoise[0]['BrownianNoise'][index]
        #use only the thermal noise at the specified frequency = default : 100 Hz 
    else:
        
        ThermalNoise_Total = ThermalNoise[0]['BrownianNoise']
    
   

    def integrand(EFI,lambda_,materialLayer,materialParams,num_points=30000):
        ### set up a function to integrate over the total elecric feild intensity as a function of depth 
        ### t
        #materialLayer:         numpy.ndarray - An array of integers where each element represents the material type for each layer in the coating stack.
        #materialParams:        dict - A dictionary containing the refractive indices for each material type. The keys are material types (as referenced in materialLayer), and each key maps to another dictionary with a key 'n' for refractive index.
        #lambda_:               float - The wavelength of light in nanometers used for calculating the layer thicknesses.
        #num_points:            int - The total number of points to represent in the array, distributed across the entire stack.
        #Returns
        #EFI/refractiveindex    numpy.ndarray - Electric feild intensity normallised to the refractive index at each point in the coating stack 

        # Initialize variables
        depths = []
        ref_indices = []
        
        # Calculate layer thicknesses
        #if np.shape(materialLayer)[0] != 1:
            
        layer_thicknesses = [lambda_ / (4 * materialParams[mat]['n']) for mat in materialLayer]
        cumulative_thickness = np.cumsum(layer_thicknesses)
            
        
        #else: 
        #    cumulative_thickness = layer_thicknesses
        #    layer_thicknesses = lambda_ / (4 * materialParams[materialLayer[0]]['n'])
            

        # Generate depth points linearly spaced across each layer
        for i, thickness in enumerate(layer_thicknesses):
            start_depth = cumulative_thickness[i] - thickness
            end_depth = cumulative_thickness[i]
            num_points_layer = int(num_points * (thickness / cumulative_thickness[-1]))

            layer_depths = np.linspace(start_depth, end_depth, num_points_layer, endpoint=False)
            layer_ref_indices = np.full(layer_depths.shape, materialParams[materialLayer[i]]['n'])

            depths.extend(layer_depths)
            ref_indices.extend(layer_ref_indices)

        # Adjust the total number of points to be exactly 30,000
        current_total_points = len(depths)
        if current_total_points != num_points:
            adjustment = num_points - current_total_points
            final_layer_depths = np.linspace(cumulative_thickness[-2], cumulative_thickness[-1], adjustment + len(depths[-adjustment:]), endpoint=False)
            final_layer_ref_indices = np.full(final_layer_depths.shape, materialParams[materialLayer[-1]]['n'])
            depths[-adjustment:] = final_layer_depths
            ref_indices[-adjustment:] = final_layer_ref_indices

        # Create final array
        stack_info_array = np.column_stack((depths, ref_indices))
        stack_info_array = pd.DataFrame(stack_info_array)

    
        return EFI/stack_info_array[1]

    
    # Total Thickness
    D = PhysicalThickness[-1]
    
    normallised_EFI = integrand(E_total,lambda_,materialLayer,materialParams,num_points=30000)
    
    depths = np.linspace(0, D, len(normallised_EFI))

    # Perform the integration using the trapezoidal rule
    E_integrated = np.trapz(normallised_EFI, depths)
    
    
    nLayer = np.zeros(len(materialLayer))
    aLayer = np.zeros(len(materialLayer))
    
    for n, mat in enumerate(materialLayer):
        nLayer[n] = materialParams[mat]['n']
        aLayer[n] = materialParams[mat]['a']
    
    nSub = 1 
    
    # Reflectivity
    R, dcdp, rbar, r = YAM_CoatingBrownian.getCoatRefl2(1, nSub, nLayer, dOpt)
    
    R = np.real(R)

    # Merit Function
    print(f"{'Parameter':<10}{'Value':<10}")
    print('-' * 20)

    print(f"{'R':<10}{R:<10.5f}")
    print(f"{'CTN':<10}{ThermalNoise_Total:<10.2e}")
    print(f"{'E':<10}{(1/100 * E_integrated):<10.2f}")
    print(f"{'D':<10}{(D):<10.2f}")

    
    
    M = w_R * (1/R) + w_T * ThermalNoise_Total + w_E * (1/E_integrated) + w_D * D
    M_scaled = w_R * (1/100 * R) + w_T * (5.92672659826259e-21 * ThermalNoise_Total) + w_E * (1/10 * E_integrated) + w_D * (1/4 * np.log10(D))
    
    return np.max(M), np.max(M_scaled)





In [219]:


wBeam = 0.062;               # 6cm beam for aLIGO 
lambda_ = 1064e-9;           # laser wavelength

Temp = 293;                  # temperature - Room temperature 
f = 100# np.logspace(0, 3, 100);  # frequencies for plotting

## set up aLIGO Coating Stack to benchmark functions 
num21 = 19  # number of 1-2 doublets (includes cap)
num34 = 0   # number of 3-4 doublets

# Using list comprehension to replicate the behavior of MATLAB's repmat
materialLayer = [1, 2] * num21 + [1, 2] * num34
materialLayer = np.array(materialLayer)

dOpt =np.ones(len(materialLayer))*0.25 
dOpt =np.array(dOpt)

materialParams = {
    1: {
        'name': 'SiO2',
        'n': 1.44,
        'a': 0,
        'alpha': 0.51e-6,
        'beta': 8e-6,
        'kappa': 1.38,
        'C': 1.64e6,
        'Y': 72e9,
        'prat': 0.17,
        'phiM': 4.6e-5,
        'k'   : 0 # right now assuming absorption of silica at 1064 is negligable. 
    },
    2: {
        'name': 'Ta2O5',
        'n': 2.07,
        'a': 2,
        'alpha': 3.6e-6,
        'beta': 14e-6,
        'kappa': 33,
        'C': 2.1e6,
        'Y': 140e9,
        'prat': 0.23,
        'phiM': 2.44e-4,
        'k'   : 1E-4 #measured by S.Tait for RLVIP tantala  at 1064nm  
    },

    999: {
        'name': 'air', 
        'n'   : 1,
        'alpha': np.NaN,
        'beta': np.NaN,
        'kappa': np.NaN,
        'C': np.NaN,
        'Y': np.NaN,
        'prat': np.NaN,
        'phiM': np.NaN
    }
}


# Calculating the merit function
E_total, _, PhysicalThickness = EFI_tmm.CalculateEFI_tmm(dOpt ,materialLayer=materialLayer, materialParams=materialParams,lambda_ =lambda_ ,t_air = 500,polarisation='p' ,plots =False)
ThermalNoise= YAM_CoatingBrownian.getCoatingThermalNoise(dOpt, materialLayer, materialParams, materialSub=1, lambda_=lambda_, f=f, wBeam=wBeam, Temp=Temp,plots =False)

## set up weights 
w_R = 100  # Weighting factor for reflectivity
w_T = 1  # Weighting factor for thermal noise
w_E = 1100  # Weighting factor for electric field intensity
w_D = 0.8  # Weighting factor for total thickness

        
merit = merit_function(dOpt,materialLayer,materialParams,w_R, w_T, w_E, w_D,E_total,PhysicalThickness,ThermalNoise)
merit

Parameter Value     
--------------------
R         1.00      
CTN       5.93e-21  
E         11.49     
D         6051.27   


(4941.971919048678, 126409.31881674263)

In [225]:
materialLayer = [1, 2] * num21 + [1, 2] * num34
materialLayer = np.array(materialLayer)

full_dOpt = np.ones(len(materialLayer)) * 0.25  # Full array of dOpt with the optimal values

full_materailLayer = materialLayer;  


dOpt = np.zeros(np.shape(materialLayer))  # Start with a single value

counter = 0 
mat_idx = np.zeros(np.shape(materialLayer))
thickness =np.zeros(np.shape(materialLayer))
Full_merit = []

# Loop through gradually revealing more of the optimal design
for mat in full_materailLayer:
   
    #print(f"idx: {counter} MatLayer: {materialLayer}")
    
    for thick_itt in thickness_range:
        thickness_range = np.linspace(0.01,0.25,100)
        dOpt[counter] = thick_itt
        mat_idx[counter] = mat
        
        filtered_dOpt    = dOpt[dOpt != 0]
        filtered_mat_idx = mat_idx[mat_idx != 0]
       # print(f"dOpt: {filtered_dOpt} MatLayer: {filtered_mat_idx}")
    
        filtered_dOpt   = np.array([filtered_dOpt]) if isinstance(filtered_dOpt, np.int64) else filtered_dOpt
        filtered_mat_idx = np.array([filtered_mat_idx]) if isinstance(filtered_mat_idx, np.int64) else filtered_mat_idx
        
        
        # Calculating the merit function
        E_total, _, PhysicalThickness = EFI_tmm.CalculateEFI_tmm(filtered_dOpt ,materialLayer=filtered_mat_idx, materialParams=materialParams,lambda_ =lambda_ ,t_air = 500,polarisation='p' ,plots =False)
    
        ThermalNoise= YAM_CoatingBrownian.getCoatingThermalNoise(filtered_dOpt ,materialLayer=filtered_mat_idx, materialParams=materialParams, materialSub=1, lambda_=lambda_, f=f, wBeam=wBeam, Temp=Temp,plots =False)
    
            
        ## set up weights 
        w_R = 100  # Weighting factor for reflectivity
        w_T = 1  # Weighting factor for thermal noise
        w_E = 10  # Weighting factor for electric field intensity
        w_D = 4  # Weighting factor for total thickness

                
        merit = merit_function(filtered_dOpt,filtered_mat_idx,materialParams,w_R, w_T, w_E, w_D,E_total,PhysicalThickness,ThermalNoise)
        
    
        Full_merit = np.append(Full_merit,merit)
    
    counter =  counter +1  
   
    
    # Print or store the merit value, and then update dOpt for the next iteration
    
   # dOpt = np.append(dOpt, 0.01)  # Increase the length of dOpt by 1 for the next iteration

Parameter Value     
--------------------
R         -0.00     
CTN       1.25e-22  
E         1.81      
D         284.72    
Parameter Value     
--------------------
R         -0.00     
CTN       1.39e-22  
E         1.81      
D         284.72    
Parameter Value     
--------------------
R         -0.00     
CTN       1.52e-22  
E         1.81      
D         284.72    
Parameter Value     
--------------------
R         -0.00     
CTN       1.64e-22  
E         1.81      
D         284.72    
Parameter Value     
--------------------
R         -0.01     
CTN       1.75e-22  
E         1.81      
D         284.72    
Parameter Value     
--------------------
R         -0.01     
CTN       1.85e-22  
E         1.81      
D         284.72    
Parameter Value     
--------------------
R         -0.01     
CTN       1.95e-22  
E         1.81      
D         284.72    
Parameter Value     
--------------------
R         -0.01     
CTN       2.05e-22  
E         1.81      
D         284

In [234]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(-Full_merit)
plt.savefig('MeritFunction_testing/Merit_first_attempt.png')



<IPython.core.display.Javascript object>

In [233]:
materialLayer = [1, 2] * num21 + [1, 2] * num34
materialLayer = np.array(materialLayer)

full_dOpt = np.ones(len(materialLayer)) * 0.25  # Full array of dOpt with the optimal values

full_materailLayer = materialLayer;  


dOpt = np.zeros(np.shape(materialLayer))  # Start with a single value

counter = 0 
mat_idx = np.zeros(np.shape(materialLayer))
thickness =np.zeros(np.shape(materialLayer))
Full_merit = []

# Loop through gradually revealing more of the optimal design
for mat in full_materailLayer:
   
    #print(f"idx: {counter} MatLayer: {materialLayer}")
    
    for thick_itt in thickness_range:
        thickness_range = np.linspace(0.01,0.25,100)
        dOpt[counter] = thick_itt
        mat_idx[counter] = mat
        
        filtered_dOpt    = dOpt[dOpt != 0]
        filtered_mat_idx = mat_idx[mat_idx != 0]
       # print(f"dOpt: {filtered_dOpt} MatLayer: {filtered_mat_idx}")
    
        filtered_dOpt   = np.array([filtered_dOpt]) if isinstance(filtered_dOpt, np.int64) else filtered_dOpt
        filtered_mat_idx = np.array([filtered_mat_idx]) if isinstance(filtered_mat_idx, np.int64) else filtered_mat_idx
        
        
    # Calculating the merit function
    E_total, _, PhysicalThickness = EFI_tmm.CalculateEFI_tmm(filtered_dOpt ,materialLayer=filtered_mat_idx, materialParams=materialParams,lambda_ =lambda_ ,t_air = 500,polarisation='p' ,plots =False)

    ThermalNoise= YAM_CoatingBrownian.getCoatingThermalNoise(filtered_dOpt ,materialLayer=filtered_mat_idx, materialParams=materialParams, materialSub=1, lambda_=lambda_, f=f, wBeam=wBeam, Temp=Temp,plots =False)

        
    ## set up weights 
    w_R = 100  # Weighting factor for reflectivity
    w_T = 1  # Weighting factor for thermal noise
    w_E = 0.1  # Weighting factor for electric field intensity
    w_D = 4  # Weighting factor for total thickness

            
    merit = merit_function(filtered_dOpt,filtered_mat_idx,materialParams,w_R, w_T, w_E, w_D,E_total,PhysicalThickness,ThermalNoise)
    

    Full_merit = np.append(Full_merit,merit)
    
    counter =  counter +1  
   
    
    # Print or store the merit value, and then update dOpt for the next iteration
    
   # dOpt = np.append(dOpt, 0.01)  # Increase the length of dOpt by 1 for the next iteration

Parameter Value     
--------------------
R         -0.35     
CTN       6.23e-22  
E         1.81      
D         284.72    
Parameter Value     
--------------------
R         0.35      
CTN       1.17e-21  
E         2.27      
D         413.22    
Parameter Value     
--------------------
R         -0.00     
CTN       1.32e-21  
E         3.15      
D         597.95    
Parameter Value     
--------------------
R         0.62      
CTN       1.69e-21  
E         3.88      
D         726.45    
Parameter Value     
--------------------
R         0.35      
CTN       1.80e-21  
E         4.75      
D         911.17    
Parameter Value     
--------------------
R         0.80      
CTN       2.13e-21  
E         5.75      
D         1039.67   
Parameter Value     
--------------------
R         0.62      
CTN       2.22e-21  
E         6.42      
D         1224.40   
Parameter Value     
--------------------
R         0.90      
CTN       2.51e-21  
E         7.29      
D         135

In [235]:
import matplotlib.pyplot as plt
import numpy as np
import time

# Initialize the plot
plt.ion()  # Turn on interactive mode
fig, ax = plt.subplots()
x = np.linspace(0, 4 * np.pi, 100)
line, = ax.plot(x, np.sin(x))

# Dynamic update loop
for t in range(100):
    # Update the data
    line.set_ydata(np.sin(x + t / 10.0))

    # Redraw the plot
    fig.canvas.draw()
    fig.canvas.flush_events()
    
    # Pause to make the updates visible
    time.sleep(0.1)


<IPython.core.display.Javascript object>